In [2]:
import pandas as pd
import numpy as np
import os
import time
import subprocess
import re
import random
import arff

from math import sqrt
from sklearn.model_selection import  train_test_split
from sklearn.model_selection import KFold
from sklearn.naive_bayes import MultinomialNB  
from sklearn.metrics import classification_report
from sklearn.metrics import coverage_error
from sklearn.metrics import label_ranking_loss
from sklearn.metrics import hamming_loss
from sklearn.metrics import accuracy_score
from sklearn import metrics
from sklearn.metrics import zero_one_loss
from sklearn.metrics import jaccard_similarity_score

# functions

In [68]:
def read_data(dataPath):
    # input: '/Volumes/Samsung_T5/research/data/ABC_news_data/obesity/'
    # read data
    data = pd.read_csv(os.path.join(dataPath,'X.csv'))
    label = pd.read_csv(os.path.join(dataPath,'Y.csv'))
    return data,label

def evaluation(y_pred, y_prob, y_true):
    
    coverage = coverage_error(y_true, y_prob)
    hamming = hamming_loss(y_true, y_pred)
    ranking_loss = label_ranking_loss(y_true, y_prob) 
    
    f1_macro = metrics.f1_score(y_true, y_pred, average='macro')
    f1_micro = metrics.f1_score(y_true, y_pred, average='micro')
    
    acc = 0
    for i in range(y_true.shape[0]):
        acc += jaccard_similarity_score(y_true.iloc[i,:], y_pred.iloc[i,:]) # jaccard_similarity_score
    acc = round(acc / y_true.shape[0],2)
    
    zero_one = zero_one_loss(y_true, y_pred) # 0-1 error 
    
    f1_each = metrics.f1_score(y_true, y_pred, average=None)
    
    performance = {"coverage_error":coverage,
                   "ranking_loss":ranking_loss,
                   "hamming_loss":hamming,
                   "f1_macro":f1_macro,
                   "f1_micro":f1_micro,
                   "Jaccard_Index":acc,
                   "zero_one_error":zero_one,
                   "f1_each_label":f1_each}
    return performance

def get_confusion_matrix(y_pred, y_test, column_names):
    """confusion matrix """
    confusion_matrix = pd.DataFrame(np.array(y_pred) - np.array(y_test), columns=column_names)
    pos = pd.DataFrame((np.array(y_pred) == np.array(y_test)) & (np.array(y_pred) == 1), columns=y_test.columns).sum(axis=0)
    neg = pd.DataFrame((np.array(y_pred) == np.array(y_test)) & (np.array(y_pred) == 0), columns=y_test.columns).sum(axis=0)
    for i in range(confusion_matrix.shape[1]): 
        name = confusion_matrix.iloc[:,i].name
        temp = confusion_matrix.iloc[:,i].value_counts()
        TP = pos[name]
        TN = neg[name]
        if 1 in temp.index:
            FP = temp[1]
        else:
            FP = 0
        if -1 in temp.index:
            FN = temp[-1]
        else:
            FN = 0

# read data



In [20]:
dataPath = '/Volumes/Samsung_T5/research/data/small_datasets/emotions/'
dataset = 'emotions'
data, label = read_data(dataPath) # read data

# get data information
n_label = label.shape[1]
n_attr = data.shape[1]
n_instance = data.shape[0]
avg_label_per_instance = label.sum(axis=1).mean()
avg_instance_per_label = label.sum(axis=0).mean()
# print data information
print("\n--- Data Information ---")
print("dataset:", dataPath)
print("number of label:",n_label)
print("number of attribute:",n_attr)
print("number of instance:",n_instance)
print("avgerage number of labels for an instance:",avg_label_per_instance)
print("avgerage number of positive instances for a label:",avg_instance_per_label,"the std:",sqrt(label.sum(axis=0).var()),"\n")

print("-- number of positive instances --")
print(label.sum(axis=0))


--- Data Information ---
dataset: /Volumes/Samsung_T5/research/data/small_datasets/emotions/
number of label: 6
number of attribute: 72
number of instance: 593
avgerage number of labels for an instance: 1.8684654300168635
avgerage number of positive instances for a label: 184.66666666666666 the std: 41.03494445794543 

-- number of positive instances --
amazed-suprised    173
happy-pleased      166
relaxing-calm      264
quiet-still        148
sad-lonely         168
angry-aggresive    189
dtype: int64


In [21]:
cooccurrence_matrix = label.T.dot(label)
np.fill_diagonal(cooccurrence_matrix.values, 0)
#cooccurrence_matrix.to_csv('/Users/jiangjunhao/Desktop/cooccurrence_matrix.csv', index=False)
cooccurrence_matrix

,amazed-suprised,happy-pleased,relaxing-calm,quiet-still,sad-lonely,angry-aggresive
amazed-suprised,0,56,13,0,10,92
happy-pleased,56,0,91,7,1,12
relaxing-calm,13,91,0,104,95,7
quiet-still,0,7,104,0,105,2
sad-lonely,10,1,95,105,0,20
angry-aggresive,92,12,7,2,20,0


# Binary Relevance 

## BR using naive Bayes classifier

In [22]:
def naiveBayes_multi_label_training(X_train, y_train):
    start = time.time()
    
    n_label = y_train.shape[1]
    classifier_list = [MultinomialNB() for i in range(n_label)]
    for i in range(n_label):
        classifier_list[i].fit(X_train,y_train.iloc[:,i])
    
    end = time.time()
    training_time = end-start
    
    return classifier_list, training_time

def naiveBayes_multi_label_testing(X_test, n_label, classifier_list):
    y_predict = pd.DataFrame()
    y_prob = pd.DataFrame()
    
    start = time.time()
    
    for i in range(n_label):
        y_predict_i = classifier_list[i].predict(X_test)
        y_predict = pd.concat([y_predict, pd.DataFrame(y_predict_i)],axis=1)
        
        y_predict_prob_i = classifier_list[i].predict_proba(X_test)[:,1]
        y_prob = pd.concat([y_prob, pd.DataFrame(y_predict_prob_i)],axis=1)
        
    end = time.time()
    testing_time = end-start
        
    return y_predict, y_prob, testing_time

def BR_test(data, label, dataPath, random_state=3071980):
    
    # data set information
    n_label = label.shape[1]
    n_attr = data.shape[1]
    n_instance = data.shape[0]
    avg_label_per_instance = label.sum(axis=1).mean()
    
    # split training and test data set
    X_train, X_test, y_train, y_test = train_test_split(data, label, test_size=0.5, random_state=random_state)
    
    print("-- test index --")
    print(X_test.index)
    
    # training
    classifier_list, training_time = naiveBayes_multi_label_training(X_train, y_train)
    
    # testing
    y_predict, y_prob, testing_time = naiveBayes_multi_label_testing(X_test, n_label, classifier_list)
    
    # evaluation
    performance = evaluation(y_predict, y_prob, y_test)
    
    # print data information
    print("--- Data Information ---")
    print("dataset:", dataPath)
    print("number of label:",n_label)
    print("number of attribute:",n_attr)
    print("number of instance:",n_instance,"\n")
    
    # get confusion matrix
    get_confusion_matrix(y_predict, y_test, y_test.columns)
    
    # print performance
    print("--- Performance ---")
    for key, value in performance.items():
        if key == "f1_each_label":
            print("\n- f1 for each label -")
            for i in range(n_label):
                print("label_"+label.columns[i],"=",round(value[i],2))
        elif key == "coverage_error":
            print(key,'=',round(value,2),"( avg_label_per_instance =",round(avg_label_per_instance,2),")")
        else:
            print(key,'=',round(value,2))
     
    
# two fold cross-validation
def two_fold_BR_test(data, label, dataPath, n_iter=5, random_state=3071980):
    
    # data set information
    n_label = label.shape[1]
    n_attr = data.shape[1]
    n_instance = data.shape[0]
    avg_label_per_instance = label.sum(axis=1).mean()
    
    # 2-fold cross validatiom
    KF=KFold(n_splits=2, shuffle=True, random_state=random_state)
    i = 0
    
    performance = {}
    for train_index,test_index in KF.split(data):
        i += 1
        
        X_train,X_test=data.iloc[train_index,:],data.iloc[test_index,:]
        y_train,y_test=label.iloc[train_index,:],label.iloc[test_index,:]
        
        print("--- kfold time="+str(i)+" ---")
        # training
        classifier_list, training_time = naiveBayes_multi_label_training(X_train, y_train)
        # testing
        y_predict, y_prob, testing_time = naiveBayes_multi_label_testing(X_test, n_label, classifier_list)
        
        # evaluation
        if performance == {}:
            performance = evaluation(y_predict, y_prob, y_test)
        else:
            performance_i = evaluation(y_predict, y_prob, y_test)
            for key, value in performance_i.items():
                performance[key] = (performance[key] + value)/2
            else:
                performance[key] = value
    
    # print data information
    print("\n--- Data Information ---")
    print("dataset:", dataPath)
    print("number of label:",n_label)
    print("number of attribute:",n_attr)
    print("number of instance:",n_instance,"\n")
    
    # print performance
    print("--- 2 fold cross-validation Performance ---")
    for key, value in performance.items():
        if key == "f1_each_label":
            continue
        elif key == "coverage_error":
            print(key,'=',round(value,2),"( avg_label_per_instance =",round(avg_label_per_instance,2),")")
        else:
            print(key,'=',round(value,2))

In [23]:
# read data
data, label = read_data(dataPath)

# train - test
print("------ Binary Relevance using Naive Bayes ------")
BR_test(data, label, dataPath,3071980)

#print("")
#print("------ two_fold Binary Relevance using Naive Bayes ------")
#two_fold_BR_test(data, label, dataPath,3071980)

------ Binary Relevance using Naive Bayes ------
-- test index --
Int64Index([305,  40, 469, 422, 166,  29, 537, 285,  57, 112,
            ...
            317,  27, 249, 551, 591,  36, 334, 480, 494, 511],
           dtype='int64', length=297)
--- Data Information ---
dataset: /Volumes/Samsung_T5/research/data/small_datasets/emotions/
number of label: 6
number of attribute: 72
number of instance: 593 

--- Performance ---
coverage_error = 2.83 ( avg_label_per_instance = 1.87 )
ranking_loss = 0.17
hamming_loss = 0.24
f1_macro = 0.63
f1_micro = 0.65
Jaccard_Index = 0.76
zero_one_error = 0.76

- f1 for each label -
label_amazed-suprised = 0.66
label_happy-pleased = 0.33
label_relaxing-calm = 0.77
label_quiet-still = 0.75
label_sad-lonely = 0.56
label_angry-aggresive = 0.69


In [31]:
# data set information
n_label = label.shape[1]
n_attr = data.shape[1]
n_instance = data.shape[0]
avg_label_per_instance = label.sum(axis=1).mean()

# split training and test data set
# X_train, X_test, y_train, y_test = train_test_split(data, label, test_size=0.5, random_state=random_state)
X_train = data[data.index.isin(index)==False]
X_test = data[data.index.isin(index)==True]
y_train = label[label.index.isin(index)==False]
y_test = label[label.index.isin(index)==True]

print("-- test index --")
print(X_test.index)

# training
print("--- start training ---\n")
classifier_list, training_time = naiveBayes_multi_label_training(X_train, y_train)

# testing
print("--- start testing ---\n")
y_predict, y_prob, testing_time = naiveBayes_multi_label_testing(X_test, n_label, classifier_list)

# get confusion matrix
get_confusion_matrix(y_predict, y_test, y_test.columns)
    
# evaluation
performance = evaluation(y_predict, y_prob, y_test)


# print data information
print("--- Data Information ---")
print("dataset:", dataPath)
print("number of label:",n_label)
print("number of attribute:",n_attr)
print("number of instance:",n_instance,"\n")

# print performance
print("--- Performance ---")
for key, value in performance.items():
    if key == "f1_each_label":
        print("\n- f1 for each label -")
        for i in range(n_label):
            print("label_"+label.columns[i],"=",round(value[i],2))
    elif key == "coverage_error":
        print(key,'=',round(value,2),"( avg_label_per_instance =",round(avg_label_per_instance,2),")")
    else:
        print(key,'=',round(value,2))

-- test index --
Int64Index([  5,   7,   8,  15,  16,  17,  21,  23,  24,  31,
            ...
            569, 572, 573, 574, 575, 576, 580, 584, 587, 592],
           dtype='int64', length=297)
--- start training ---

--- start testing ---

--- Data Information ---
dataset: /Volumes/Samsung_T5/research/data/small_datasets/emotions/
number of label: 6
number of attribute: 72
number of instance: 593 

--- Performance ---
coverage_error = 2.76 ( avg_label_per_instance = 1.87 )
ranking_loss = 0.17
hamming_loss = 0.23
f1_macro = 0.63
f1_micro = 0.65
Jaccard_Index = 0.77
zero_one_error = 0.73

- f1 for each label -
label_amazed-suprised = 0.57
label_happy-pleased = 0.38
label_relaxing-calm = 0.75
label_quiet-still = 0.76
label_sad-lonely = 0.55
label_angry-aggresive = 0.74


## BR using ESKDB

In [32]:
def get_arff(word_occurrence, label_matrix, savePath): # get attributes
    for z in range(len(label_matrix.columns)):
        attributes=[(word_occurrence.columns[i],list(map(str,sorted(word_occurrence.iloc[:,i].unique())))) for i in range(len(word_occurrence.columns))]
        attributes.append(('label_'+label_matrix.columns[z],['0', '1']))

        data=[]
        i = 0
        while i < label_matrix.shape[0]:
            attr_data = [str(j) for j in list(word_occurrence.iloc[i,:])]
            label_data = [str(label_matrix.iloc[i,z])]
            row_data = attr_data+label_data
            data.append(row_data) 
            i+=1
        # set obj
        obj = {
           'description': u'',
           'relation': 'relation',
           'attributes': attributes,
           'data': data,
        }
        arff_data = arff.dumps(obj)
        w_file = open(savePath+label_matrix.columns[z]+".arff", "w")
        w_file.write(arff_data)
        w_file.close()

def run_eskdb(dataPath, resultFile, k, l, e, i):
    command = "./run_eskdb.sh "+resultFile+" "+k+" "+i+" "+l+" "+e+" "+dataPath
    subprocess.call("cd /Volumes/Samsung_T5/research/programme/research_python/", shell=True)
    print(command)
    return subprocess.call(command, shell=True)

def get_result(resultPath):

    y_pred = pd.DataFrame()
    y_true = pd.DataFrame()
    y_prob = pd.DataFrame()
    names = []
    for file in os.listdir(resultPath):
        with open(os.path.join(resultPath,file), 'r') as f:
            try:
                lines = f.readlines()
            except:
                print(file)
            else:
                names.append(file[:-4])
                pred = []
                true = []
                prob = []
                train_time_total = 0
                test_time_total = 0
                error_marco = 0
                for line in lines:
                    if line.startswith('pred'):
                        pred.append(int(re.search('pred :\t(.)',line).group(1)))
                        true.append(int(re.search('true :\t(.)',line).group(1)))
                        prob.append(float(re.search('prob :\t(.*)',line).group(1)))
                    elif line.startswith('RSME'):
                        rsme = float(re.search('RSME :\t\t(.*)',line).group(1))
                    elif line.startswith('Error'):
                        error = float(re.search('Error :\t\t(.*)',line).group(1))
                    elif line.startswith("Training time"):
                        train_time = float(re.search('Training time :\s{1,}(.*)',line).group(1))
                        train_time_total = train_time_total + train_time
                    elif line.startswith("Testing time"):
                        test_time = float(re.search('Testing time :\s{1,}(.*)',line).group(1))
                        test_time_total = test_time_total + test_time
                    elif line.startswith("["):
                        para = line
                    elif line.startswith("test0Indexes"):
                        index = list(map(int,re.search('test0Indexes: {(.*)}',line).group(1).split(', ')))

                y_pred = pd.concat([y_pred,pd.DataFrame(pred)],axis=1)
                y_true = pd.concat([y_true,pd.DataFrame(true)],axis=1)
                y_prob = pd.concat([y_prob,pd.DataFrame(prob)],axis=1)
    y_pred.columns = names
    y_true.columns = names
    y_pred.index = index
    y_true.index = index
    print(para)
    print("number of label:", y_pred.shape[1])
    print("training time:",train_time_total)
    print("testing time:",test_time_total)
    return y_pred,y_true,y_prob,index

In [28]:
# read data
data, label = read_data(dataPath)

# get arff files

savePath = "/Users/jiangjunhao/Desktop/test/"+dataset+'/'
os.mkdir(savePath)
get_arff(data,label,savePath)

resultFile = dataset+'_k5_e20_i5000'
k = '5'
i = '5000'
l = '2'
e = '20'

run_eskdb(savePath, resultFile, k, l, e, i)

./run_eskdb.sh emotions_k5_e20_i5000 5 5000 2 20 /Users/jiangjunhao/Desktop/test/emotions/


0

In [33]:
resultPath = '/Volumes/Samsung_T5/research/programme/ESKDB_HDP/result/'+resultFile
y_pred,y_true,y_prob,index = get_result(resultPath)

performance = evaluation(y_pred=y_pred, y_true=y_true, y_prob=y_prob)
    
# print performance
print("--- Performance ---")
for key, value in performance.items():
    if key == "f1_each_label":
        print("\n- f1 for each label -")
        for i in range(n_label):
            print("label_"+label.columns[i],"=",round(value[i],2))
    elif key == "coverage_error":
        print(key,'=',round(value,2),"( avg_label_per_instance =",round(avg_label_per_instance,2),")")
    else:
        print(key,'=',round(value,2))

[-t, /Users/jiangjunhao/Desktop/test/emotions//sad-lonely.arff, -S, ESKDB, -K, 5, -I, 5000, -L, 2, -E, 20, -V, -M]

number of label: 6
training time: 550.0
testing time: 240.0
--- Performance ---
coverage_error = 2.67 ( avg_label_per_instance = 1.87 )
ranking_loss = 0.15
hamming_loss = 0.19
f1_macro = 0.64
f1_micro = 0.67
Jaccard_Index = 0.81
zero_one_error = 0.69

- f1 for each label -
label_amazed-suprised = 0.64
label_happy-pleased = 0.75
label_relaxing-calm = 0.32
label_quiet-still = 0.73
label_sad-lonely = 0.78
label_angry-aggresive = 0.6


# Classifier Chain

## CC using naive Bayes

In [34]:
def naiveBayes_multi_label_training(X_train, y_train):
    start = time.time()
    
    n_label = y_train.shape[1]
    
    order = random.sample(list(range(n_label)),n_label) # get orders
    
    classifier_list = [MultinomialNB() for i in range(n_label)] # create a classifier chain
    
    for i in range(n_label):
        if i == 0:
            classifier_list[i].fit(X_train,y_train.iloc[:, order[i]])
        else:
            X_train = pd.concat([X_train, y_train.iloc[:,order[i-1]]],axis=1) # put the previous label into attribute space
            classifier_list[i].fit(X_train,y_train.iloc[:,order[i]])

    end = time.time()
    training_time = end-start
    
    return classifier_list, training_time, order

def naiveBayes_multi_label_testing(X_test, n_label, classifier_list, order):
    y_predict = pd.DataFrame(index=X_test.index)
    y_prob = pd.DataFrame(index=X_test.index)
    y_true = pd.DataFrame(index=X_test.index)
    
    start = time.time()
    
    for i in range(n_label):
        y_predict_i = classifier_list[i].predict(X_test)
        y_predict = pd.concat([y_predict, pd.DataFrame(y_predict_i,index=X_test.index)],axis=1)

        y_predict_prob_i = classifier_list[i].predict_proba(X_test)[:,1]
        y_prob = pd.concat([y_prob, pd.DataFrame(y_predict_prob_i,index=X_test.index)],axis=1)

        X_test = pd.concat([X_test, pd.DataFrame(y_predict_i,index=X_test.index)],axis=1,ignore_index=True) # put the previous label into attribute space

    end = time.time()
    testing_time = end-start
        
    return y_predict, y_prob, testing_time

def CC_test(data, label, dataPath, random_state=3071980):
    
    # data set information
    n_label = label.shape[1]
    n_attr = data.shape[1]
    n_instance = data.shape[0]
    avg_label_per_instance = label.sum(axis=1).mean()
    
    # split training and test data set
    X_train, X_test, y_train, y_test = train_test_split(data, label, test_size=0.2, random_state=random_state)
    
    # training
    print("--- start training ---\n")
    classifier_list, training_time, order = naiveBayes_multi_label_training(X_train, y_train)
    
    # testing
    print("--- start testing ---\n")
    y_predict, y_prob, testing_time = naiveBayes_multi_label_testing(X_test, n_label, classifier_list, order)
    
    # evaluation
    y_test = y_test.iloc[:,order]
    performance = evaluation(y_predict, y_prob, y_test)
    
    # print data information
    print("--- Data Information ---")
    print("dataset:", dataPath)
    print("number of label:",n_label)
    print("number of attribute:",n_attr)
    print("number of instance:",n_instance,"\n")
    
    # print orders
    print("\n--- Order of the chain ---")
    print(label.columns[order])
    print("")
    
    # print performance
    print("--- Performance ---")
    for key, value in performance.items():
        if key == "f1_each_label":
            print("\n- f1 for each label -")
            for i in range(n_label):
                print("label_"+label.columns[i],"=",round(value[i],2))
        elif key == "coverage_error":
            print(key,'=',round(value,2),"( avg_label_per_instance =",round(avg_label_per_instance,2),")")
        else:
            print(key,'=',round(value,2))
            
def ECC_test(data, label, dataPath, random_state=3071980, ensemble = 5):
    
    # data set information
    n_label = label.shape[1]
    n_attr = data.shape[1]
    n_instance = data.shape[0]
    avg_label_per_instance = label.sum(axis=1).mean()
    
    # split training and test data set
    X_train, X_test, y_train, y_test = train_test_split(data, label, test_size=0.5, random_state=random_state)
    
    # ensemble
    y_pred_ensemble = pd.DataFrame(np.zeros(y_test.shape),columns=y_test.columns, index=y_test.index)
    y_prob_ensemble = pd.DataFrame(np.zeros(y_test.shape),columns=y_test.columns, index=y_test.index)
    for i in range(ensemble):
        # training
        #print("--- start training ---\n")
        classifier_list, training_time, order = naiveBayes_multi_label_training(X_train, y_train)

        # print orders
        print("Order of the chain:",label.columns[order])

        # testing
        #print("--- start testing ---\n")
        y_predict, y_prob, testing_time = naiveBayes_multi_label_testing(X_test, n_label, classifier_list, order)

        y_predict.columns = label.columns[order]
        y_prob.columns = label.columns[order]
        y_predict = y_predict[label.columns]
        y_prob = y_prob[label.columns]

        y_pred_ensemble = y_pred_ensemble + y_predict
        y_prob_ensemble = y_prob_ensemble + y_prob

    y_pred_ensemble = (((y_pred_ensemble / ensemble) >= 0.5)*1).astype('int')
    y_prob_ensemble = y_prob_ensemble / ensemble 
    
    # evaluation
    performance = evaluation(y_pred_ensemble, y_prob_ensemble, y_test)
    
    # print data information
    print("--- Data Information ---")
    print("dataset:", dataPath)
    print("number of label:",n_label)
    print("number of attribute:",n_attr)
    print("number of instance:",n_instance,"\n")
    
    # print orders
    print("\n--- Order of the chain ---")
    print(label.columns[order])
    print("")
    
    # print performance
    print("--- Performance ---")
    for key, value in performance.items():
        if key == "f1_each_label":
            print("\n- f1 for each label -")
            for i in range(n_label):
                print("label_"+label.columns[i],"=",round(value[i],2))
        elif key == "coverage_error":
            print(key,'=',round(value,2),"( avg_label_per_instance =",round(avg_label_per_instance,2),")")
        else:
            print(key,'=',round(value,2))

In [35]:
# read data
data, label = read_data(dataPath)

# train - test
print("------ Binary Relevance using Naive Bayes ------")
ECC_test(data, label, dataPath, 3071980, 1)

------ Binary Relevance using Naive Bayes ------
Order of the chain: Index(['happy-pleased', 'amazed-suprised', 'sad-lonely', 'quiet-still',
       'relaxing-calm', 'angry-aggresive'],
      dtype='object')
--- Data Information ---
dataset: /Volumes/Samsung_T5/research/data/small_datasets/emotions/
number of label: 6
number of attribute: 72
number of instance: 593 


--- Order of the chain ---
Index(['happy-pleased', 'amazed-suprised', 'sad-lonely', 'quiet-still',
       'relaxing-calm', 'angry-aggresive'],
      dtype='object')

--- Performance ---
coverage_error = 2.84 ( avg_label_per_instance = 1.87 )
ranking_loss = 0.17
hamming_loss = 0.23
f1_macro = 0.63
f1_micro = 0.66
Jaccard_Index = 0.77
zero_one_error = 0.73

- f1 for each label -
label_amazed-suprised = 0.65
label_happy-pleased = 0.33
label_relaxing-calm = 0.78
label_quiet-still = 0.76
label_sad-lonely = 0.58
label_angry-aggresive = 0.71


In [36]:
# data set information
n_label = label.shape[1]
n_attr = data.shape[1]
n_instance = data.shape[0]
avg_label_per_instance = label.sum(axis=1).mean()

# split training and test data set
# X_train, X_test, y_train, y_test = train_test_split(data, label, test_size=0.5, random_state=random_state)

X_train = data[data.index.isin(index)==False]
X_test = data[data.index.isin(index)==True]
y_train = label[label.index.isin(index)==False]
y_test = label[label.index.isin(index)==True]

print("-- test index --")
print(X_test.index)

# training
print("--- start training ---\n")
classifier_list, training_time, order = naiveBayes_multi_label_training(X_train, y_train)

# testing
print("--- start testing ---\n")
y_predict, y_prob, testing_time = naiveBayes_multi_label_testing(X_test, n_label, classifier_list, order)

# evaluation
y_test = y_test.iloc[:,order]
performance = evaluation(y_predict, y_prob, y_test)

# print data information
print("--- Data Information ---")
print("dataset:", dataPath)
print("number of label:",n_label)
print("number of attribute:",n_attr)
print("number of instance:",n_instance,"\n")

# print orders
print("\n--- Order of the chain ---")
print(label.columns[order])
print("")


# print performance
print("--- Performance ---")
for key, value in performance.items():
    if key == "f1_each_label":
        print("\n- f1 for each label -")
        for i in range(n_label):
            print("label_"+label.columns[i],"=",round(value[i],2))
    elif key == "coverage_error":
        print(key,'=',round(value,2),"( avg_label_per_instance =",round(avg_label_per_instance,2),")")
    else:
        print(key,'=',round(value,2))

-- test index --
Int64Index([  5,   7,   8,  15,  16,  17,  21,  23,  24,  31,
            ...
            569, 572, 573, 574, 575, 576, 580, 584, 587, 592],
           dtype='int64', length=297)
--- start training ---

--- start testing ---

--- Data Information ---
dataset: /Volumes/Samsung_T5/research/data/small_datasets/emotions/
number of label: 6
number of attribute: 72
number of instance: 593 


--- Order of the chain ---
Index(['amazed-suprised', 'happy-pleased', 'relaxing-calm', 'sad-lonely',
       'angry-aggresive', 'quiet-still'],
      dtype='object')

--- Performance ---
coverage_error = 2.73 ( avg_label_per_instance = 1.87 )
ranking_loss = 0.17
hamming_loss = 0.23
f1_macro = 0.63
f1_micro = 0.66
Jaccard_Index = 0.77
zero_one_error = 0.72

- f1 for each label -
label_amazed-suprised = 0.57
label_happy-pleased = 0.38
label_relaxing-calm = 0.77
label_quiet-still = 0.59
label_sad-lonely = 0.75
label_angry-aggresive = 0.74


## Ensemble CC using naive Bayes (E = 10)

In [37]:
# read data
data, label = read_data(dataPath)

# train - test
print("------ Binary Relevance using Naive Bayes ------")
ECC_test(data, label, dataPath, random_state=3071980, ensemble=10)

------ Binary Relevance using Naive Bayes ------
Order of the chain: Index(['relaxing-calm', 'quiet-still', 'amazed-suprised', 'sad-lonely',
       'angry-aggresive', 'happy-pleased'],
      dtype='object')
Order of the chain: Index(['sad-lonely', 'quiet-still', 'happy-pleased', 'angry-aggresive',
       'relaxing-calm', 'amazed-suprised'],
      dtype='object')
Order of the chain: Index(['amazed-suprised', 'angry-aggresive', 'sad-lonely', 'relaxing-calm',
       'quiet-still', 'happy-pleased'],
      dtype='object')
Order of the chain: Index(['happy-pleased', 'angry-aggresive', 'relaxing-calm', 'sad-lonely',
       'quiet-still', 'amazed-suprised'],
      dtype='object')
Order of the chain: Index(['angry-aggresive', 'happy-pleased', 'sad-lonely', 'relaxing-calm',
       'quiet-still', 'amazed-suprised'],
      dtype='object')
Order of the chain: Index(['amazed-suprised', 'sad-lonely', 'happy-pleased', 'relaxing-calm',
       'quiet-still', 'angry-aggresive'],
      dtype='object')
Ord

In [38]:
# read data
data, label = read_data(dataPath)

ensemble=10

# data set information
n_label = label.shape[1]
n_attr = data.shape[1]
n_instance = data.shape[0]
avg_label_per_instance = label.sum(axis=1).mean()

X_train = data[data.index.isin(index)==False]
X_test = data[data.index.isin(index)==True]
y_train = label[label.index.isin(index)==False]
y_test = label[label.index.isin(index)==True]

print("-- test index --")
print(X_test.index)

# ensemble
y_pred_ensemble = pd.DataFrame(np.zeros(y_test.shape),columns=y_test.columns, index=y_test.index)
y_prob_ensemble = pd.DataFrame(np.zeros(y_test.shape),columns=y_test.columns, index=y_test.index)
for i in range(ensemble):
    # training
    #print("--- start training ---\n")
    classifier_list, training_time, order = naiveBayes_multi_label_training(X_train, y_train)

    # print orders
    print("Order of the chain:",label.columns[order])

    # testing
    #print("--- start testing ---\n")
    y_predict, y_prob, testing_time = naiveBayes_multi_label_testing(X_test, n_label, classifier_list, order)

    y_predict.columns = label.columns[order]
    y_prob.columns = label.columns[order]
    y_predict = y_predict[label.columns]
    y_prob = y_prob[label.columns]

    y_pred_ensemble = y_pred_ensemble + y_predict
    y_prob_ensemble = y_prob_ensemble + y_prob

y_pred_ensemble = (((y_pred_ensemble / ensemble) >= 0.5)*1).astype('int')
y_prob_ensemble = y_prob_ensemble / ensemble 

# evaluation
performance = evaluation(y_pred_ensemble, y_prob_ensemble, y_test)

# print data information
print("--- Data Information ---")
print("dataset:", dataPath)
print("number of label:",n_label)
print("number of attribute:",n_attr)
print("number of instance:",n_instance,"\n")

# print orders
print("\n--- Order of the chain ---")
print(label.columns[order])
print("")

# get confusion matrix
get_confusion_matrix(y_pred_ensemble, y_test, y_test.columns)

# print performance
print("--- Performance ---")
for key, value in performance.items():
    if key == "f1_each_label":
        print("\n- f1 for each label -")
        for i in range(n_label):
            print("label_"+label.columns[i],"=",round(value[i],2))
    elif key == "coverage_error":
        print(key,'=',round(value,2),"( avg_label_per_instance =",round(avg_label_per_instance,2),")")
    else:
        print(key,'=',round(value,2))

-- test index --
Int64Index([  5,   7,   8,  15,  16,  17,  21,  23,  24,  31,
            ...
            569, 572, 573, 574, 575, 576, 580, 584, 587, 592],
           dtype='int64', length=297)
Order of the chain: Index(['relaxing-calm', 'amazed-suprised', 'quiet-still', 'angry-aggresive',
       'sad-lonely', 'happy-pleased'],
      dtype='object')
Order of the chain: Index(['amazed-suprised', 'sad-lonely', 'angry-aggresive', 'relaxing-calm',
       'quiet-still', 'happy-pleased'],
      dtype='object')
Order of the chain: Index(['happy-pleased', 'relaxing-calm', 'amazed-suprised', 'angry-aggresive',
       'quiet-still', 'sad-lonely'],
      dtype='object')
Order of the chain: Index(['happy-pleased', 'quiet-still', 'angry-aggresive', 'sad-lonely',
       'amazed-suprised', 'relaxing-calm'],
      dtype='object')
Order of the chain: Index(['relaxing-calm', 'amazed-suprised', 'quiet-still', 'angry-aggresive',
       'sad-lonely', 'happy-pleased'],
      dtype='object')
Order of the c

## Ensemble CC using naive Bayes (E = 50)

In [39]:
# read data
data, label = read_data(dataPath)

# train - test
print("------ Binary Relevance using Naive Bayes ------")
ECC_test(data, label, dataPath, random_state=3071980, ensemble=50)

------ Binary Relevance using Naive Bayes ------
Order of the chain: Index(['happy-pleased', 'sad-lonely', 'relaxing-calm', 'quiet-still',
       'amazed-suprised', 'angry-aggresive'],
      dtype='object')
Order of the chain: Index(['happy-pleased', 'relaxing-calm', 'sad-lonely', 'angry-aggresive',
       'quiet-still', 'amazed-suprised'],
      dtype='object')
Order of the chain: Index(['sad-lonely', 'amazed-suprised', 'relaxing-calm', 'angry-aggresive',
       'happy-pleased', 'quiet-still'],
      dtype='object')
Order of the chain: Index(['amazed-suprised', 'angry-aggresive', 'happy-pleased', 'quiet-still',
       'relaxing-calm', 'sad-lonely'],
      dtype='object')
Order of the chain: Index(['sad-lonely', 'relaxing-calm', 'angry-aggresive', 'amazed-suprised',
       'happy-pleased', 'quiet-still'],
      dtype='object')
Order of the chain: Index(['quiet-still', 'amazed-suprised', 'angry-aggresive', 'sad-lonely',
       'relaxing-calm', 'happy-pleased'],
      dtype='object')
Ord

In [40]:
# read data
data, label = read_data(dataPath)

ensemble=50

# data set information
n_label = label.shape[1]
n_attr = data.shape[1]
n_instance = data.shape[0]
avg_label_per_instance = label.sum(axis=1).mean()

X_train = data[data.index.isin(index)==False]
X_test = data[data.index.isin(index)==True]
y_train = label[label.index.isin(index)==False]
y_test = label[label.index.isin(index)==True]

print("-- test index --")
print(X_test.index)

# ensemble
y_pred_ensemble = pd.DataFrame(np.zeros(y_test.shape),columns=y_test.columns, index=y_test.index)
y_prob_ensemble = pd.DataFrame(np.zeros(y_test.shape),columns=y_test.columns, index=y_test.index)
for i in range(ensemble):
    # training
    #print("--- start training ---\n")
    classifier_list, training_time, order = naiveBayes_multi_label_training(X_train, y_train)

    # print orders
    print("Order of the chain:",label.columns[order])

    # testing
    #print("--- start testing ---\n")
    y_predict, y_prob, testing_time = naiveBayes_multi_label_testing(X_test, n_label, classifier_list, order)

    y_predict.columns = label.columns[order]
    y_prob.columns = label.columns[order]
    y_predict = y_predict[label.columns]
    y_prob = y_prob[label.columns]

    y_pred_ensemble = y_pred_ensemble + y_predict
    y_prob_ensemble = y_prob_ensemble + y_prob

y_pred_ensemble = (((y_pred_ensemble / ensemble) >= 0.5)*1).astype('int')
y_prob_ensemble = y_prob_ensemble / ensemble 

# evaluation
performance = evaluation(y_pred_ensemble, y_prob_ensemble, y_test)


# print data information
print("--- Data Information ---")
print("dataset:", dataPath)
print("number of label:",n_label)
print("number of attribute:",n_attr)
print("number of instance:",n_instance,"\n")

# print orders
print("\n--- Order of the chain ---")
print(label.columns[order])
print("")

# get confusion matrix
get_confusion_matrix(y_pred_ensemble, y_test, y_test.columns)

# print performance
print("--- Performance ---")
for key, value in performance.items():
    if key == "f1_each_label":
        print("\n- f1 for each label -")
        for i in range(n_label):
            print("label_"+label.columns[i],"=",round(value[i],2))
    elif key == "coverage_error":
        print(key,'=',round(value,2),"( avg_label_per_instance =",round(avg_label_per_instance,2),")")
    else:
        print(key,'=',round(value,2))

-- test index --
Int64Index([  5,   7,   8,  15,  16,  17,  21,  23,  24,  31,
            ...
            569, 572, 573, 574, 575, 576, 580, 584, 587, 592],
           dtype='int64', length=297)
Order of the chain: Index(['quiet-still', 'sad-lonely', 'relaxing-calm', 'happy-pleased',
       'angry-aggresive', 'amazed-suprised'],
      dtype='object')
Order of the chain: Index(['quiet-still', 'sad-lonely', 'relaxing-calm', 'happy-pleased',
       'angry-aggresive', 'amazed-suprised'],
      dtype='object')
Order of the chain: Index(['sad-lonely', 'amazed-suprised', 'happy-pleased', 'angry-aggresive',
       'quiet-still', 'relaxing-calm'],
      dtype='object')
Order of the chain: Index(['quiet-still', 'angry-aggresive', 'relaxing-calm', 'amazed-suprised',
       'sad-lonely', 'happy-pleased'],
      dtype='object')
Order of the chain: Index(['quiet-still', 'amazed-suprised', 'relaxing-calm', 'sad-lonely',
       'happy-pleased', 'angry-aggresive'],
      dtype='object')
Order of the c

## CC using ESKDB


In [52]:
# prepare data
def csv_to_arff(X, label_i, savePath):
    # get attributes
    attributes=[(X.columns[i],['0', '1']) for i in range(len(X.columns))]
    attributes.append(('label_'+label_i.name,['0', '1']))

    data=[]
    i = 0
    while i < len(label_i):
        attr_data = [str(j) for j in list(X.iloc[i,:])]
        label_data = [str(label_i[i])]
        row_data = attr_data+label_data
        data.append(row_data) 
        i+=1
    # set obj
    obj = {
       'description': u'',
       'relation': 'relation',
       'attributes': attributes,
       'data': data,
    }
    arff_data = arff.dumps(obj)
    w_file = open(savePath+label_i.name+".arff", "w")
    w_file.write(arff_data)
    w_file.close()

def get_arff(X, label, savePath):
    
    n_label = label.shape[1]
    # get orders
    order = random.sample(list(range(n_label)),n_label) # get orders
    
    #  get all arff files, one for each label
    for i in range(n_label):
        label_i = label.iloc[:,order[i]]
        print("--Running label:",label_i.name)
        csv_to_arff(X, label_i, savePath)
        
        label_i.name = 'label_' + label_i.name
        X = pd.concat([X, label_i], axis=1)
    print("--finished getting arff files")
    return order

def run_eskdb(label_arff, resultFile, k, l, e, i):
    command = "./run_ECC.sh "+resultFile+" "+k+" "+i+" "+l+" "+e+" "+label_arff
    subprocess.call("cd /Volumes/Samsung_T5/research/programme/research_python/", shell=True)
    #print(command)
    return subprocess.call(command, shell=True)

def get_result(resultPath):

    y_pred = pd.DataFrame()
    y_true = pd.DataFrame()
    y_prob = pd.DataFrame()
    names = []
    for file in os.listdir(resultPath):
        with open(os.path.join(resultPath,file), 'r') as f:
            try:
                lines = f.readlines()
            except:
                print(file)
            else:
                names.append(file[:-4])
                pred = []
                true = []
                prob = []
                train_time_total = 0
                test_time_total = 0
                error_marco = 0
                for line in lines:
                    if line.startswith('pred'):
                        pred.append(int(re.search('pred :\t(.)',line).group(1)))
                        true.append(int(re.search('true :\t(.)',line).group(1)))
                        prob.append(float(re.search('prob :\t(.*)',line).group(1)))
                    elif line.startswith('RSME'):
                        rsme = float(re.search('RSME :\t\t(.*)',line).group(1))
                    elif line.startswith('Error'):
                        error = float(re.search('Error :\t\t(.*)',line).group(1))
                    elif line.startswith("Training time"):
                        train_time = float(re.search('Training time :\s{1,}(.*)',line).group(1))
                        train_time_total = train_time_total + train_time
                    elif line.startswith("Testing time"):
                        test_time = float(re.search('Testing time :\s{1,}(.*)',line).group(1))
                        test_time_total = test_time_total + test_time
                    elif line.startswith("["):
                        para = line
                    elif line.startswith("test0Indexes"):
                        index = list(map(int,re.search('test0Indexes: {(.*)}',line).group(1).split(', ')))

                y_pred = pd.concat([y_pred,pd.DataFrame(pred)],axis=1)
                y_true = pd.concat([y_true,pd.DataFrame(true)],axis=1)
                y_prob = pd.concat([y_prob,pd.DataFrame(prob)],axis=1)
    y_pred.columns = names
    y_true.columns = names
    y_prob.columns = names
    print(para)
    print("number of label:", y_pred.shape[1])
    print("training time:",train_time_total)
    print("testing time:",test_time_total)
    return y_pred,y_true,y_prob,index

In [57]:
# read data
data, label = read_data(dataPath)
n_label = label.shape[1]

resultFile = dataset+'_k5_e20_i5000_CC'

# train and test on the first label
savePath = "/Users/jiangjunhao/Desktop/test/"+dataset+'/'

# data set information
n_label = label.shape[1]
n_attr = data.shape[1]
n_instance = data.shape[0]
avg_label_per_instance = label.sum(axis=1).mean()

# get orders
order = random.sample(list(range(n_label)),n_label) # get orders


## prepare data, get the arff file
for i in range(n_label):
    label_i = label.iloc[:,order[i]]
    #print("--Running label:",label_i.name)
    csv_to_arff(data, label_i, savePath)

    # run eskdb
    label_arff = os.path.join(savePath,label_i.name+'.arff')
    k = '5'
    i = '5000'
    l = '2'
    e = '20'

    run_eskdb(label_arff, resultFile, k, l, e, i)

    result = os.path.join("/Volumes/Samsung_T5/research/programme/ESKDB_HDP/result",resultFile, label_i.name+'.txt')
    with open(result, 'r') as f:
        try:
            lines = f.readlines()
        except:
            print(file)
        else:
            pred = []
            for line in lines:
                if line.startswith('pred'):
                    pred.append(int(re.search('pred :\t(.)',line).group(1)))
                elif line.startswith("test0Indexes"):
                    index = list(map(int,re.search('test0Indexes: {(.*)}',line).group(1).split(', ')))

            label.loc[index,label_i.name] = pred
            temp = label.loc[:,label_i.name]
            temp.name = 'label_'+label_i.name
            data = pd.concat([data, label.loc[:,label_i.name]],axis=1)

# get result
resultPath = '/Volumes/Samsung_T5/research/programme/ESKDB_HDP/result/'+resultFile+'/'
y_pred,y_true,y_prob,index = get_result(resultPath)
performance = evaluation(y_pred=y_pred, y_true=y_true, y_prob=y_prob)

# print orders:
print(label.columns[order])

# get confusion matrix
print("\n--- Confusion matrix ---")
get_confusion_matrix(y_pred, y_true, y_pred.columns)

# print performance
print("--- Performance ---")
for key, value in performance.items():
    if key == "f1_each_label":
        print("\n- f1 for each label -")
        for i in range(n_label):
            print("label_"+label.columns[i],"=",round(value[i],2))
    elif key == "coverage_error":
        print(key,'=',round(value,2),"( avg_label_per_instance =",round(avg_label_per_instance,2),")")
    else:
        print(key,'=',round(value,2))

[-t, /Users/jiangjunhao/Desktop/test/emotions/angry-aggresive.arff, -S, ESKDB, -K, 5, -I, 5000, -L, 2, -E, 20, -V, -M]

number of label: 6
training time: 459.0
testing time: 298.0
Index(['amazed-suprised', 'relaxing-calm', 'quiet-still', 'sad-lonely',
       'happy-pleased', 'angry-aggresive'],
      dtype='object')

--- Confusion matrix ---
--- Performance ---
coverage_error = 2.67 ( avg_label_per_instance = 1.87 )
ranking_loss = 0.15
hamming_loss = 0.2
f1_macro = 0.68
f1_micro = 0.69
Jaccard_Index = 0.8
zero_one_error = 0.66

- f1 for each label -
label_amazed-suprised = 0.65
label_happy-pleased = 0.78
label_relaxing-calm = 0.77
label_quiet-still = 0.6
label_sad-lonely = 0.51
label_angry-aggresive = 0.77


## Ensemble CC using ESKDB(E=10)

In [76]:
# read data
data, label = read_data(dataPath)
n_label = label.shape[1]

# data set information
n_label = label.shape[1]
n_attr = data.shape[1]
n_instance = data.shape[0]
avg_label_per_instance = label.sum(axis=1).mean()

# ensemble
ensemble = 10

y_pred_ensemble = pd.DataFrame(np.zeros((int(label.shape[0]/2+1),label.shape[1])),columns=label.columns)
y_prob_ensemble = pd.DataFrame(np.zeros((int(label.shape[0]/2+1),label.shape[1])),columns=label.columns)
    
    
for i in range(ensemble):

    # read data
    data, label = read_data(dataPath)
    n_label = label.shape[1]

    # data set information
    n_label = label.shape[1]
    n_attr = data.shape[1]
    n_instance = data.shape[0]
    avg_label_per_instance = label.sum(axis=1).mean()

    # get orders
    order = random.sample(list(range(n_label)),n_label) # get orders

    # train and test on the first label
    savePath = "/Users/jiangjunhao/Desktop/test/"+dataset+'/'

    ## prepare data, get the arff file
    for i in range(n_label):
        label_i = label.iloc[:,order[i]]
        print("--Running label:",label_i.name)
        csv_to_arff(data, label_i, savePath)

        # run eskdb
        label_arff = os.path.join(savePath,label_i.name+'.arff')
        k = '5'
        i = '5000'
        l = '2'
        e = '20'

        run_eskdb(label_arff, resultFile, k, l, e, i)

        result = os.path.join("/Volumes/Samsung_T5/research/programme/ESKDB_HDP/result",resultFile, label_i.name+'.txt')
        with open(result, 'r') as f:
            try:
                lines = f.readlines()
            except:
                print(file)
            else:
                pred = []
                for line in lines:
                    if line.startswith('pred'):
                        pred.append(int(re.search('pred :\t(.)',line).group(1)))
                    elif line.startswith("test0Indexes"):
                        index = list(map(int,re.search('test0Indexes: {(.*)}',line).group(1).split(', ')))

                label.loc[index,label_i.name] = pred
                temp = label.loc[:,label_i.name]
                temp.name = 'label_'+label_i.name
                data = pd.concat([data, label.loc[:,label_i.name]],axis=1)

    # get result
    resultPath = '/Volumes/Samsung_T5/research/programme/ESKDB_HDP/result/'+resultFile+'/'
    y_pred,y_true,y_prob,index = get_result(resultPath)
    print(y_prob.tail())
    y_pred.columns = label.columns
    y_prob.columns = label.columns
    
    y_pred_ensemble = y_pred_ensemble + y_pred
    y_prob_ensemble = y_prob_ensemble + y_prob
    
    
y_pred_ensemble = (((y_pred_ensemble / ensemble) >= 0.5)*1).astype('int')
y_prob_ensemble = y_prob_ensemble / ensemble 

--Running label: happy-pleased
--Running label: sad-lonely
--Running label: relaxing-calm
--Running label: amazed-suprised
--Running label: quiet-still
--Running label: angry-aggresive
._amazed-suprised.txt
[-t, /Users/jiangjunhao/Desktop/test/emotions/angry-aggresive.arff, -S, ESKDB, -K, 5, -I, 5000, -L, 2, -E, 20, -V, -M]

number of label: 6
training time: 450.0
testing time: 234.0
     amazed-suprised  relaxing-calm  quiet-still  sad-lonely  happy-pleased  \
292         0.849749       0.007767     0.002300    0.011501       0.310735   
293         0.102385       0.852809     0.530466    0.411915       0.143949   
294         0.175936       0.392429     0.011528    0.109340       0.421825   
295         0.814646       0.039006     0.011906    0.218215       0.338874   
296         0.137750       0.905489     0.102268    0.002421       0.609740   

     angry-aggresive  
292         0.961474  
293         0.000992  
294         0.645188  
295         0.824088  
296         0.000266  


--Running label: angry-aggresive
--Running label: happy-pleased
--Running label: relaxing-calm
--Running label: sad-lonely
--Running label: amazed-suprised
._amazed-suprised.txt
[-t, /Users/jiangjunhao/Desktop/test/emotions/angry-aggresive.arff, -S, ESKDB, -K, 5, -I, 5000, -L, 2, -E, 20, -V, -M]

number of label: 6
training time: 414.0
testing time: 184.0
     amazed-suprised  relaxing-calm  quiet-still  sad-lonely  happy-pleased  \
292         0.898242       0.004310     0.003277    0.029145       0.106669   
293         0.015948       0.819179     0.542008    0.653643       0.023763   
294         0.071542       0.755985     0.042273    0.006769       0.616787   
295         0.758356       0.016345     0.015221    0.075903       0.152291   
296         0.197321       0.973736     0.142737    0.001967       0.800303   

     angry-aggresive  
292         0.823596  
293         0.011193  
294         0.374005  
295         0.728147  
296         0.012914  


In [78]:
performance = evaluation(y_pred=y_pred_ensemble, y_true=y_true, y_prob=y_prob_ensemble)

# print orders:
print(label.columns[order])

# get confusion matrix
print("\n--- Confusion matrix ---")
get_confusion_matrix(y_pred, y_true, y_pred.columns)

# print performance
print("--- Performance ---")
for key, value in performance.items():
    if key == "f1_each_label":
        print("\n- f1 for each label -")
        for i in range(n_label):
            print("label_"+label.columns[i],"=",round(value[i],2))
    elif key == "coverage_error":
        print(key,'=',round(value,2),"( avg_label_per_instance =",round(avg_label_per_instance,2),")")
    else:
        print(key,'=',round(value,2))

Index(['quiet-still', 'angry-aggresive', 'happy-pleased', 'relaxing-calm',
       'sad-lonely', 'amazed-suprised'],
      dtype='object')

--- Confusion matrix ---
--- Performance ---
coverage_error = 2.63 ( avg_label_per_instance = 1.87 )
ranking_loss = 0.15
hamming_loss = 0.2
f1_macro = 0.68
f1_micro = 0.69
Jaccard_Index = 0.8
zero_one_error = 0.69

- f1 for each label -
label_amazed-suprised = 0.63
label_happy-pleased = 0.79
label_relaxing-calm = 0.76
label_quiet-still = 0.61
label_sad-lonely = 0.51
label_angry-aggresive = 0.77


# Result

`
**Performance of BR_naive Bayes**
coverage_error = 13.6 ( avg_label_per_instance = 3.25 )
ranking_loss = 0.13
hamming_loss = 0.1
f1_macro = 0.26
f1_micro = 0.4
Jaccard_Index = 0.26
zero_one_error = 0.98
`

`
**Performance of BR_ESKDB**
coverage_error = 14.76 ( avg_label_per_instance = 3.25 )
ranking_loss = 0.15
hamming_loss = 0.08
f1_macro = 0.15
f1_micro = 0.33
jaccard_index = 0.23
zero_one_error = 0.94
`

`
**Performance of Ensemble Classifier Chain using naive Bayes**
coverage_error = 13.63 ( avg_label_per_instance = 3.25 )
ranking_loss = 0.13
hamming_loss = 0.11
f1_macro = 0.26
f1_micro = 0.38
Jaccard_Index = 0.25
zero_one_error = 0.97
`

`
**Performance of Ensemble Classifier Chain using ESKDB(E=2)**
coverage_error = 15.87 ( avg_label_per_instance = 3.25 )
ranking_loss = 0.16
hamming_loss = 0.08
f1_macro = 0.22
f1_micro = 0.38
Jaccard_Index = 0.25
zero_one_error = 0.95
`


recall 体现了分类模型H对正样本的识别能力，recall 越高，说明模型对正样本的识别能力越强.

precision 体现了模型对负样本的区分能力，precision越高，说明模型对负样本的区分能力越强。F1-score 是两者的综合。F1-score 越高，说明分类模型越稳健。